In [1]:
import torch
import torch.nn as nn
import numpy as np
torch.__version__

'1.0.1.post2'

In [3]:
data=np.loadtxt("german.data-numeric")

In [4]:
n,l=data.shape
for j in range(l-1):
    meanVal=np.mean(data[:,j])
    stdVal=np.std(data[:,j])
    data[:,j]=(data[:,j]-meanVal)/stdVal

In [5]:
np.random.shuffle(data)
train_data=data[:900,:l-1]
train_lab=data[:900,l-1]-1
test_data=data[900:,:l-1]
test_lab=data[900:,l-1]-1

In [11]:
# 定义模型
class LR(nn.Module):
    def __init__(self):
        super(LR,self).__init__()
        self.fc=nn.Linear(24,2)
    def forward(self,x):
        out=self.fc(x)
        out=torch.sigmoid(out)
        return out

In [7]:
# 测试准确率
def test(pred,lab):
    t=pred.max(-1)[1]==lab
    return torch.mean(t.float())

In [20]:
net=LR()
criterion=nn.CrossEntropyLoss()  # 使用CrossEntroyLoss损失
optm=torch.optim.Adam(net.parameters()) # Adam优化
epochs=10000#训练1000次

In [21]:
# 训练
for i in range(epochs):
    net.train()  #指定模型为战训练模式，计算梯度
    x=torch.from_numpy(train_data).float()
    y=torch.from_numpy(train_lab).long()
    y_hat=net(x)
    loss=criterion(y_hat,y) # 计算损失
    optm.zero_grad() #前一步的损失清零
    loss.backward() #反向传播
    optm.step() # 优化
    if(i+1)%100==0:
        net.eval() # 指定网络计算模式
        test_in=torch.from_numpy(test_data).float()
        test_l=torch.from_numpy(test_lab).long()
        test_out=net(test_in)
        accu=test(test_out,test_l) # 计算准确率
        print("Epoch:{},loss:{:.4f},Accuracy:{:.2f}".format(i+1,loss.item(),accu))

Epoch:100,loss:0.6437,Accuracy:0.66
Epoch:200,loss:0.6174,Accuracy:0.70
Epoch:300,loss:0.6006,Accuracy:0.72
Epoch:400,loss:0.5888,Accuracy:0.72
Epoch:500,loss:0.5798,Accuracy:0.74
Epoch:600,loss:0.5728,Accuracy:0.75
Epoch:700,loss:0.5670,Accuracy:0.76
Epoch:800,loss:0.5622,Accuracy:0.77
Epoch:900,loss:0.5581,Accuracy:0.78
Epoch:1000,loss:0.5544,Accuracy:0.79
Epoch:1100,loss:0.5512,Accuracy:0.79
Epoch:1200,loss:0.5484,Accuracy:0.79
Epoch:1300,loss:0.5459,Accuracy:0.79
Epoch:1400,loss:0.5437,Accuracy:0.79
Epoch:1500,loss:0.5417,Accuracy:0.79
Epoch:1600,loss:0.5399,Accuracy:0.79
Epoch:1700,loss:0.5382,Accuracy:0.79
Epoch:1800,loss:0.5366,Accuracy:0.80
Epoch:1900,loss:0.5352,Accuracy:0.80
Epoch:2000,loss:0.5339,Accuracy:0.80
Epoch:2100,loss:0.5326,Accuracy:0.79
Epoch:2200,loss:0.5315,Accuracy:0.79
Epoch:2300,loss:0.5304,Accuracy:0.80
Epoch:2400,loss:0.5294,Accuracy:0.80
Epoch:2500,loss:0.5284,Accuracy:0.80
Epoch:2600,loss:0.5275,Accuracy:0.81
Epoch:2700,loss:0.5266,Accuracy:0.81
Epoch:2800

In [22]:
# cuda版本
device=torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU
net=LR().to(device)
optm=torch.optim.Adam(net.parameters())
def cuda_test(pred,lab):
    pred,lab=pred.to(device),lab.to(device)
    t=pred.max(-1)[1]==lab
    return torch.mean(t.float())
# 训练
for i in range(epochs):
    net.train()
    x=torch.from_numpy(train_data).float()
    y=torch.from_numpy(train_lab).long()
    x,y=x.to(device),y.to(device)
    y_hat=net(x)
    loss=criterion(y_hat,y)
    optm.zero_grad() #前一步的损失清零
    loss.backward()
    optm.step()
    if (i+1)%100==0:
        net.eval()
        test_in=torch.from_numpy(test_data).float()
        test_l=torch.from_numpy(test_lab).long()
        test_in,test_l=test_in.to(device),test_l.to(device)
        test_out=net(test_in)
        accu=cuda_test(test_out,test_l)
        print("CUDA Eporch:{},Loss:{:.4f},Accuracy:{:.2f}".format(i+1,loss.item(),accu))

CUDA Eporch:100,Loss:0.6403,Accuracy:0.68
CUDA Eporch:200,Loss:0.6148,Accuracy:0.72
CUDA Eporch:300,Loss:0.5978,Accuracy:0.73
CUDA Eporch:400,Loss:0.5857,Accuracy:0.74
CUDA Eporch:500,Loss:0.5766,Accuracy:0.78
CUDA Eporch:600,Loss:0.5695,Accuracy:0.78
CUDA Eporch:700,Loss:0.5638,Accuracy:0.78
CUDA Eporch:800,Loss:0.5590,Accuracy:0.79
CUDA Eporch:900,Loss:0.5551,Accuracy:0.79
CUDA Eporch:1000,Loss:0.5516,Accuracy:0.79
CUDA Eporch:1100,Loss:0.5487,Accuracy:0.79
CUDA Eporch:1200,Loss:0.5461,Accuracy:0.79
CUDA Eporch:1300,Loss:0.5438,Accuracy:0.79
CUDA Eporch:1400,Loss:0.5417,Accuracy:0.79
CUDA Eporch:1500,Loss:0.5398,Accuracy:0.79
CUDA Eporch:1600,Loss:0.5381,Accuracy:0.79
CUDA Eporch:1700,Loss:0.5366,Accuracy:0.79
CUDA Eporch:1800,Loss:0.5351,Accuracy:0.80
CUDA Eporch:1900,Loss:0.5338,Accuracy:0.80
CUDA Eporch:2000,Loss:0.5326,Accuracy:0.80
CUDA Eporch:2100,Loss:0.5315,Accuracy:0.79
CUDA Eporch:2200,Loss:0.5305,Accuracy:0.79
CUDA Eporch:2300,Loss:0.5295,Accuracy:0.79
CUDA Eporch:2400,Los